In [1]:
import sys

sys.path.append('../')

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from probabilistic_covshift.constants.automl_constants import AutoMLConfig as AutoMLConfig
from probabilistic_covshift.constants.automl_constants import H2OServerInfo as H2OServerInfo
from probabilistic_covshift.constants.main_constants import OriginFeatures as OriginFeatures
from probabilistic_covshift.constants.main_constants import WeightFeatures as WeightFeatures
from probabilistic_covshift.probabilistic_classification_covshift import ProbabilisticClassification

In [2]:
spark = SparkSession.builder.appName('main').master('local[4]').getOrCreate()

In [3]:
source_df = spark.createDataFrame([
    ('A', 38.9, 40.0, 55, 'E', 10.0), ('A', 88.9, 50.0, 15, 'E', 20.0), ('B', 18.9, 40.0, 55, 'P', 30.0),
    ('A', 38.9, 50.0, 15, 'E', 10.0), ('B', 48.9, 40.0, 55, 'P', 20.0), ('B', 58.9, 50.0, 15, 'E', 30.0), 
    ('A', 38.9, 40.0, 55, 'P', 10.0), ('B', 98.9, 50.0, 15, 'E', 20.0), ('A', 38.9, 40.0, 55, 'E', 10.0),
    ('A', 88.9, 50.0, 15, 'E', 20.0), ('B', 18.9, 40.0, 55, 'P', 30.0), ('A', 38.9, 50.0, 15, 'E', 10.0),
    ('B', 48.9, 40.0, 55, 'P', 20.0), ('B', 58.9, 50.0, 15, 'E', 30.0), ('A', 38.9, 40.0, 55, 'P', 10.0),
    ('B', 98.9, 50.0, 15, 'E', 20.0), ('A', 38.9, 40.0, 55, 'E', 10.0), ('A', 88.9, 50.0, 15, 'E', 20.0),
    ('B', 18.9, 40.0, 55, 'P', 30.0), ('A', 38.9, 50.0, 15, 'E', 10.0), ('B', 48.9, 40.0, 55, 'P', 20.0),
    ('B', 58.9, 50.0, 15, 'E', 30.0), ('A', 38.9, 40.0, 55, 'P', 10.0), ('B', 98.9, 50.0, 15, 'E', 20.0),
    ('A', 38.9, 40.0, 55, 'E', 10.0), ('A', 88.9, 50.0, 15, 'E', 20.0), ('B', 18.9, 40.0, 55, 'P', 30.0),
    ('A', 38.9, 50.0, 15, 'E', 10.0), ('B', 48.9, 40.0, 55, 'P', 20.0), ('B', 58.9, 50.0, 15, 'E', 30.0), 
    ('A', 38.9, 40.0, 55, 'P', 10.0), ('B', 98.9, 50.0, 15, 'E', 20.0), ('A', 38.9, 40.0, 55, 'E', 10.0),
    ('A', 88.9, 50.0, 15, 'E', 20.0), ('B', 18.9, 40.0, 55, 'P', 30.0), ('A', 38.9, 50.0, 15, 'E', 10.0),
    ('B', 48.9, 40.0, 55, 'P', 20.0), ('B', 58.9, 50.0, 15, 'E', 30.0), ('A', 38.9, 40.0, 55, 'P', 10.0),
    ('B', 98.9, 50.0, 15, 'E', 20.0), ('A', 38.9, 40.0, 55, 'E', 10.0), ('A', 88.9, 50.0, 15, 'E', 20.0),
    ('B', 18.9, 40.0, 55, 'P', 30.0), ('A', 38.9, 50.0, 15, 'E', 10.0), ('B', 48.9, 40.0, 55, 'P', 20.0),
    ('B', 58.9, 50.0, 15, 'E', 30.0), ('A', 38.9, 40.0, 55, 'P', 10.0), ('B', 98.9, 50.0, 15, 'E', 20.0)], 
    ['col_a', 'col_b', 'col_c', 'col_d', 'col_e', 'col_f'])

In [4]:
target_df = spark.createDataFrame([
    ('A', 48.9, 40.0, 55, 'E', 10.0), ('A', 58.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 40.0, 55, 'P', 30.0),
    ('A', 48.9, 50.0, 15, 'E', 10.0), ('B', 58.9, 40.0, 55, 'P', 10.0), ('B', 38.9, 50.0, 15, 'E', 30.0),
    ('A', 48.9, 40.0, 55, 'P', 10.0), ('B', 58.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 50.0, 15, 'E', 20.0),
    ('A', 48.9, 40.0, 55, 'E', 10.0), ('A', 58.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 40.0, 55, 'P', 30.0),
    ('A', 48.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 40.0, 55, 'P', 10.0), ('B', 38.9, 50.0, 15, 'E', 30.0),
    ('A', 48.9, 40.0, 55, 'P', 10.0), ('B', 38.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 50.0, 15, 'E', 20.0),
    ('A', 58.9, 40.0, 55, 'E', 10.0), ('A', 38.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 40.0, 55, 'P', 30.0),
    ('A', 58.9, 50.0, 15, 'E', 10.0), ('B', 148.9, 140.0, 55, 'P', 10.0), ('B', 38.9, 50.0, 15, 'E', 30.0),
    ('A', 58.9, 40.0, 55, 'P', 110.0), ('B', 198.9, 150.0, 15, 'E', 10.0), ('B', 38.9, 50.0, 15, 'E', 20.0),
    ('A', 58.9, 40.0, 55, 'E', 110.0), ('A', 38.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 140.0, 55, 'P', 30.0),
    ('A', 58.9, 50.0, 15, 'E', 110.0), ('B', 38.9, 40.0, 55, 'P', 10.0), ('B', 38.9, 150.0, 15, 'E', 30.0),
    ('A', 58.9, 40.0, 55, 'P', 110.0), ('B', 38.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 150.0, 15, 'E', 20.0)], 
    ['col_a', 'col_b', 'col_c', 'col_d', 'col_e', 'col_f'])

In [5]:
automl_conf = {
    AutoMLConfig.DATA: {
        AutoMLConfig.LABEL_COL: OriginFeatures.ORIGIN,
        AutoMLConfig.WEIGHT_COL: WeightFeatures.WEIGHT,
        AutoMLConfig.BASE_TABLE_PATH: 'data/base_table.parquet',
        AutoMLConfig.WEIGHT_PATH: 'data/weight.csv'
    },
    AutoMLConfig.SERVER_CONN_INFO: {
        H2OServerInfo.IP: 'localhost',
        H2OServerInfo.PORT: '54321'
    },
    AutoMLConfig.CROSS_VAL: {
        AutoMLConfig.FOLD_COL: "fold",
        AutoMLConfig.NFOLDS: 8,
    },
    AutoMLConfig.MODELING: {
        AutoMLConfig.MAX_RUNTIME_SECS: 3600,
        AutoMLConfig.MAX_MODELS: 10,
        AutoMLConfig.STOPPING_METRIC: 'logloss',
        AutoMLConfig.SORT_METRIC: 'logloss'
    },
    AutoMLConfig.EXCLUDE_ALGOS: [
        "StackedEnsemble",
        "DeepLearning"
    ],
    AutoMLConfig.MODEL: {
        AutoMLConfig.BEST_MODEL_PATH: 'data/model/'
    },
    AutoMLConfig.SEED: 23
}

In [6]:
pc = ProbabilisticClassification(source_df, target_df, automl_conf)
pc.run()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 hour 45 mins
H2O cluster timezone:,Asia/Jakarta
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,2 months and 5 days
H2O cluster name:,H2O_from_python_albertus_kelvin_xeikza
H2O cluster total nodes:,1
H2O cluster free memory:,3.533 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Base table inferred column types: {'col_b': 'real', 'col_c': 'int', 'col_d': 'int', 'col_f': 'int', 'origin': 'enum', 'row_id': 'int'}


INFO:probabilistic_covshift.automl.trainer:Base table inferred column types: {'col_b': 'real', 'col_c': 'int', 'col_d': 'int', 'col_f': 'int', 'origin': 'enum', 'row_id': 'int'}


AutoML progress: |
19:51:51.192: Fold column fold will be used for cross-validation. nfolds parameter will be ignored.

██████████████
19:52:00.250: Skipping training of model GBM_5_AutoML_20200325_195151 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200325_195151.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 84.0.


██████████████████████████████████████████| 100%


...


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,89.0,89.0,19129.0,3.0,6.0,5.123596,5.0,20.0,12.505618


Loaded model: 


INFO:probabilistic_covshift.automl.predictor:Loaded model: 


gbm prediction progress: |████████████████████████████████████████████████| 100%
Export File progress: |███████████████████████████████████████████████████| 100%
